### ESMA 4016
### Feature Selection en Clasificacion Supervisada
### Edgar Acuna

Uso de la liberia Sklearn de ASU y de skrebate para feaure selection

I-Usando la prueba de Chi-Square para ver si hay independenia entre la feature y la clase. 

De preferencia los datos deben estar discretizados y ademas con dos clases es mejor

Personalmente no la recomiendo porque la priueba de Chisquare es aproximada y no muy robusta


In [30]:
#Chi-Square usando la libreia de la ASU
import pandas as pd
import numpy as np
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2, SelectPercentile, f_classif, mutual_info_classif
# load data
url= "http://academic.uprm.edu/eacuna/diabetes.dat"
names = ['preg', 'plas', 'pres', 'skin', 'test', 'mass', 'pedi', 'age', 'class']
data = pd.read_table(url, names=names,header=None)
y=data['class']
X=data.iloc[:,0:8]
y1=y.as_matrix()
X1=X.as_matrix()
# feature extraction
test = SelectKBest(score_func=chi2, k=4)
fit = test.fit(X1, y1)
# summarize scores
numpy.set_printoptions(precision=3)
print(fit.scores_)
features= fit.transform(X)
# summarize selected features
print(features)

[  111.52   1411.887    17.605    53.108  2175.565   127.669     5.393
   181.304]
[[ 148.     0.    33.6   50. ]
 [  85.     0.    26.6   31. ]
 [ 183.     0.    23.3   32. ]
 ..., 
 [ 121.   112.    26.2   30. ]
 [ 126.     0.    30.1   47. ]
 [  93.     0.    30.4   23. ]]


Comentario: Las tres, mejores variables con la prueba de Chi-square son test,plas y age

II-Usando la prueba de estadistica de F para comparar dos o mas grupos

Se puede usar la libreria sciki-learn o la libreria de la ASU

In [31]:
# Selecion  de Features usando los p-values de la F-test como score
# We use the default selection function: the 10% most significant features
selector = SelectPercentile(f_classif, percentile=10)
selector.fit(X, y)
scores = -np.log10(selector.pvalues_)
scores /= scores.max()
print(scores)

[ 0.221  1.     0.027  0.034  0.084  0.378  0.14   0.253]


Comentario: Las tres, mejores variables con la prueba de F son plas,mass y age

In [32]:
from skfeature.function.statistical_based import f_score
scoref = f_score.f_score(X_train, y_train)
idx = f_score.feature_ranking(scoref)
print(scoref,idx)

(array([  2.750e+00,   1.609e+00,   9.016e-05,   8.180e+00,   6.089e+00,
         1.128e+00]), array([3, 4, 0, 1, 5, 2], dtype=int64))


Comentario: las tres variables mas importantes con la prueba de F son: plas, mass y age casi empatado con preg

III- Usando Mutual Information (Entropia)

Libreria de la ASU


In [33]:
# Univariate feature selection with Mutual Information
# We use the default selection function: the 10% most significant features
scores = mutual_info_classif(X,y)
scores /= scores.max()
print(scores)

[ 0.46   1.     0.268  0.227  0.387  0.746  0.084  0.309]


Comentario: Las tres variables usando el criterio de Mutual Information  son: plas, mass y age

IV-Usando ReliefF de la libreria skrebate

In [34]:
import pandas as pd
import numpy as np
from skrebate import ReliefF
from sklearn.model_selection import train_test_split

#Leyendo el conjunto de datos pima-diabetes 
url= "http://academic.uprm.edu/eacuna/diabetes.dat"
names = ['preg', 'plas', 'pres', 'skin', 'test', 'mass', 'pedi', 'age', 'class']
data = pd.read_table(url, names=names,header=None)
df=data.drop('class',axis=1)
#Normalizando las predictoras
df_norm=(df - df.min()) / (df.max() - df.min())
features, labels = df_norm.values, data['class'].values

In [35]:
X_train, X_test, y_train, y_test = train_test_split(features, labels)

fs = ReliefF(n_neighbors=10)
fs.fit(X_train, y_train)

for feature_name, feature_score in zip(df.columns,fs.feature_importances_):
    print (feature_name, feature_score)


('preg', 37.264705882352914)
('plas', 90.865482233502561)
('pres', 10.909836065573774)
('skin', 43.090909090909093)
('test', 22.227541371158406)
('mass', 41.479880774962709)
('pedi', 26.99380871050386)
('age', 41.233333333333306)


Comentario: las tres variables mas importantes con el ReliefF son: plas, skin y mass

In [36]:
from skfeature.function.similarity_based import reliefF
score_relief=reliefF.reliefF(X_train,y_train)
feat=reliefF.feature_ranking(score_relief)
print(score_relief,feat)

(array([  3.071,  15.215,   3.544,  10.824,   3.297,  11.106,   6.532,
         7.023]), array([1, 5, 3, 7, 6, 2, 4, 0], dtype=int64))


Comentario: las tres variables mas importantes con el RelieF son: plas, mass y preg

In [39]:
import Orange
voting = Orange.data.Table("bupa")

def print_best_3(ma):
    for m in ma[:3]:
        print "%5.3f %s" % (m[1], m[0])

print 'Feature scores for best three features (scored individually):'
meas = Orange.feature.scoring.Relief(k=20, m=50)
mr = [ (a.name, meas(a, voting)) for a in voting.domain.attributes]
mr.sort(key=lambda x: -x[1]) #sort decreasingly by the score
print_best_3(mr)

Feature scores for best three features (scored individually):
0.015 alkphos
0.013 drinks
0.009 sgpt


In [40]:
# load data
url= "http://academic.uprm.edu/eacuna/bupa.dat"
names = ['mcv', 'alkphos', 'sgpt', 'aspar', 'gammagt', 'drinks', 'class']
data = pd.read_table(url, names=names,header=None)
y=data['class']
X=data.iloc[:,0:6]
y1=y.as_matrix()
X1=X.as_matrix()
#Normalizando las predictoras
features, labels = X.values, data['class'].values

In [41]:
X_train, X_test, y_train, y_test = train_test_split(features, labels)

In [42]:
score_relief=reliefF.reliefF(X_train,y_train)
feat=reliefF.feature_ranking(score_relief)
print(score_relief,feat)

(array([  49.4,   98.6,  249. ,  124.4,  626. ,   -5.4]), array([4, 2, 3, 1, 0, 5], dtype=int64))


In [ ]:
El Rielief de ASU recomienda gmmagt sgpt y aspar como las mejoras predicotas

In [43]:
fs = ReliefF(n_neighbors=10)
fs.fit(X_train, y_train)

for feature_name, feature_score in zip(data.columns,fs.feature_importances_):
    print (feature_name, feature_score)


('mcv', 0.98000000000000353)
('alkphos', 0.21171171171171305)
('sgpt', 6.8708609271523198)
('aspar', 9.2402597402597362)
('gammagt', 13.940068493150688)
('drinks', 1.6499999999999995)


In [ ]:
El relielf de skrebate recomienda gammagt, aspar y sgpt 

In [66]:
url='http://academic.uprm.edu/eacuna/landsat.txt'
data = pd.read_table(url, header=None,delim_whitespace=True)
y=data.iloc[:,36]
X=data.iloc[:,0:37]
y1=y.as_matrix()
X1=X.as_matrix()
features, labels = X.values, y.values
X_train, X_test, y_train, y_test = train_test_split(features, labels)

In [67]:
score_relief=reliefF.reliefF(X_train,y_train)
feat=reliefF.feature_ranking(score_relief)
print(score_relief,feat)

(array([  941747.744,  1279733.562,   902156.332,  1059044.258,
         891007.836,  1209213.365,   843318.948,  1015684.606,
         941338.702,  1298755.499,   871575.744,  1105771.587,
         961920.502,  1254938.283,   876021.335,  1071337.556,
         945309.496,  1287183.944,   842435.611,  1008633.472,
        1001780.645,  1377210.983,   782792.16 ,  1039137.431,
         968576.684,  1324333.508,   867002.867,  1040189.761,
         924347.479,  1312179.194,   823315.111,   991393.79 ,
         969701.117,  1391778.929,   857372.205,  1087866.692,   222420.787]), array([33, 21, 25, 29,  9, 17,  1, 13,  5, 11, 35, 15,  3, 27, 23,  7, 19,
       20, 31, 32, 24, 12, 16,  0,  8, 28,  2,  4, 14, 10, 26, 34,  6, 18,
       30, 22, 36], dtype=int64))
